In [ ]:

def stations_clusters_on_map(clustered_stations: pd.DataFrame):
    # Create a map centered at the mean coordinates of all stations
    center_lat = clustered_stations['latitude'].mean()
    center_lon = clustered_stations['longitude'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Add markers for each station, colored by cluster
    colors = ['#%06x' % np.random.randint(0, 0xFFFFFF) for _ in range(clustered_stations["cluster"].nunique())]
    for _, row in clustered_stations.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            color=colors[int(row["cluster"])] if row["cluster"] != -1 else 'gray',
            fill=True,
            popup=f"Cluster {row["cluster"]}"
        ).add_to(m)

    """# Add markers for cluster centers
    for i, center in cluster_centers.iterrows():
        folium.Marker(
            location=[center['latitude'], center['longitude']],
            icon=folium.Icon(color='black', icon='info-sign'),
            popup=f'Cluster Center {i}'
        ).add_to(m)"""
    display(m)

stations_clusters_on_map(clustered_stations_dbscan)